# Demo DataSMART script for manual input.

This is an example notebook showing usage of `ManualDBActionWithSchema`.

To run it, first install action using commands like

~~~
./install_action.py ~/datasmart-test demo/school_grade_input
~~~

Where `~/datasmart-test` is the **installation folder**

Then change directory to installation folder of action (`~/datasmart-test`) in this case, and type in the `export` and `activate` commands on top of `start_demo_school_grade_input.sh`, plus starting Jupyter.

~~~
. activate datasmart
export PYTHONPATH=/Users/yimengzh/Research/datasmart:$PYTHONPATH
jupyter notebook
~~~

The example assumes that you work with a MongoDB database without authentication, at `localhost:27017`, and you have all privileges for MongoDB collection `demo/school_grade_input`. If this is not the case, change `config/core/db/config.json` in the installation folder, and adjust your MongoDB privileges accordingly.

## clean up

To make things simpler, we first drop the collection, and remove the 

In [1]:
from datasmart.core.db import DB, DBContextManager
from datasmart.actions.demo.school_grade_input import SchoolGradeInputAction, subjectlist
import os.path
import os
from faker import Factory
fake = Factory.create()
fake.seed(seed=0)
from datetime import datetime, time
from random import randint, seed, choice, random
seed(a=0)
from datasmart.core.util.datetime import (datetime_to_datetime_local,
                                          util_config, datetime_local_to_rfc3339_local,
                                         rfc3339_to_datetime, 
                                         datetime_local_to_local, datetime_to_datetime_utc)
from copy import deepcopy
import pandas as pd
print('current time zone set in local `config/core/util/config.json`: {}'.format(util_config['timezone']))

current time zone set in local `config/core/util/config.json`: US/Eastern


In [2]:
db_context_manager = DBContextManager(DB())
def cleanup_db():
    with db_context_manager as db_instance:
        db_instance.client_instance['demo']['school_grade_input'].drop()
        # make sure there's nothing in it.
        assert not list(db_instance.client_instance['demo']['school_grade_input'].find())
        
def cleanup_prepare_files():
    a = SchoolGradeInputAction()
    prepare_result_name = a.prepare_result_name
    query_template_name = a.query_template_name
    if os.path.exists(prepare_result_name):
        os.remove(prepare_result_name)
    if os.path.exists(query_template_name):
        os.remove(query_template_name)

In [3]:
cleanup_db()
cleanup_prepare_files()

## structure of a record

`SchoolGradeInputAction` is a minimal example showing how manually collected information is handled by `ManualDBActionWithSchema`. It has the following fields.

* `timestamp`
* `first_name`
* `last_name`
* `subject` one of "math", "english", "music", "drawing".
* `score` 0-100, integer.
* `notes` (optional).

Below is a function to randomly generate a valid record, for later usage.

In [4]:
first_name_list = ('Emma', 'Olivia', 'Liam', 'Sophia', 'Emily')
last_name_list = ('Jobs', 'Banana', 'Apple', 'Bush')


def generate_datetime():
# generate a time between 8:00am - 8:59:59 pm from 2000, 1, 1, to 2010, 12, 31.
# to avoid DST issue.
    date_random = fake.date_time_between_dates(datetime_start=datetime(2000,1,1), datetime_end=datetime(2010,12,31)).date()
    time_random = time(randint(8, 20), randint(0, 59), randint(0, 59))
    datetime_random = datetime.combine(date_random, time_random)
    return datetime_local_to_rfc3339_local(datetime_to_datetime_local(datetime_random))



def generate_record():
    record_this = {}
    record_this['first_name'] = choice(first_name_list)
    record_this['last_name'] = choice(last_name_list)
    record_this['subject'] = choice(subjectlist)
    record_this['score'] = randint(0, 100)
    record_this['timestamp'] = generate_datetime()
    if random() > 0.8:
        record_this['notes'] = fake.sentence()
    return record_this

In [5]:
# generate 100 records, show first 20
records = [generate_record() for _ in range(100)]
pd_records = pd.DataFrame(records[:20], columns=('timestamp', 'first_name', 'last_name', 'subject', 'score', 'notes'))
pd_records

,timestamp,first_name,last_name,subject,score,notes
0,2006-07-21T20:53:19-04:00,Emma,Apple,drawing,51,Animi facilis quaerat placeat necessitatibus i...
1,2006-02-02T10:48:06-05:00,Liam,Banana,english,36,NaN
2,2009-12-04T19:04:57-05:00,Liam,Banana,music,12,Quae deserunt id deserunt consequatur sit.
3,2004-04-05T14:20:39-04:00,Liam,Bush,math,45,NaN
4,2009-01-22T12:03:51-05:00,Olivia,Bush,drawing,66,Dignissimos ipsa consequuntur aut vitae iure o...
5,2006-01-07T20:42:40-05:00,Emma,Jobs,drawing,90,NaN
6,2007-05-21T13:45:55-04:00,Sophia,Apple,english,93,NaN
7,2005-05-18T20:34:28-04:00,Emily,Banana,english,18,NaN
8,2010-05-23T16:18:45-04:00,Liam,Bush,math,38,NaN
9,2010-11-23T09:38:51-05:00,Liam,Banana,music,56,NaN


## insert one record

The most usual scenario is you insert one record at a time. Usually, this is done through the interactive API of `ManualDBActionWithSchema`, where users are asked to save the record in a `json` file inside installation folder. However, it's inconvenient for the purpose of demonstration here, as it involves interaction. Therefore, we use a (sort of) batch API to demonstrate this.

In [6]:
# here record to use is the first one.
config_single = deepcopy(SchoolGradeInputAction().config)
config_single['batch_records'] = records[:1]
action_this = SchoolGradeInputAction(config_single)
action_this.run()

custom info from this action follows.




this is the DataSMART action for saving grade data for an exam.
Please modify school_grade_input_template.json to your need





done 1/1!
done!
remember to rm actions.demo.school_grade_input.prepare_result.p and actions.demo.school_grade_input.query_template.py if you want to start over for new action!


In [7]:
# check what we have in the database
with db_context_manager as db_instance:
    result_single = list(db_instance.client_instance['demo']['school_grade_input'].find())
# replace time with correct local time.
def get_correct_timezone(result_this):
    result_correct_timezone = []
    for x in result_this:
        x_new = deepcopy(x)
        x_new['timestamp'] = datetime_local_to_local(datetime_to_datetime_utc(x_new['timestamp']))
        result_correct_timezone.append(x_new)
    return result_correct_timezone
result_single_pd_raw = pd.DataFrame(result_single)
result_single_pd_correct_tz = pd.DataFrame(get_correct_timezone(result_single))

In [8]:
result_single_pd_raw

,_id,first_name,last_name,notes,score,subject,timestamp
0,57cf0f47aa9f2e116b0fd1e7,Emma,Apple,Animi facilis quaerat placeat necessitatibus i...,51,drawing,2006-07-22 00:53:19


In [9]:
result_single_pd_correct_tz

,_id,first_name,last_name,notes,score,subject,timestamp
0,57cf0f47aa9f2e116b0fd1e7,Emma,Apple,Animi facilis quaerat placeat necessitatibus i...,51,drawing,2006-07-21 20:53:19-04:00


In [10]:
# notice that running it again will say it's already done.
action_this = SchoolGradeInputAction(config_single)
action_this.run()

the action has been finished!


In [11]:
# we can revoke it and then run()
action_this = SchoolGradeInputAction(config_single)
action_this.revoke()
action_this.run()

done clearing!
custom info from this action follows.




this is the DataSMART action for saving grade data for an exam.
Please modify school_grade_input_template.json to your need





done 1/1!
done!
remember to rm actions.demo.school_grade_input.prepare_result.p and actions.demo.school_grade_input.query_template.py if you want to start over for new action!


In [12]:
with db_context_manager as db_instance:
    result_single = list(db_instance.client_instance['demo']['school_grade_input'].find())
result_single_pd_correct_tz_2 = pd.DataFrame(get_correct_timezone(result_single))
result_single_pd_correct_tz_2

,_id,first_name,last_name,notes,score,subject,timestamp
0,57cf0f47aa9f2e116b0fd1e7,Emma,Apple,Animi facilis quaerat placeat necessitatibus i...,51,drawing,2006-07-21 20:53:19-04:00


In [13]:
# notice that here, _id is exactly the same. This won't be the case if we clear the files.
# (well, you also need to create a new class for action; otherwise, the preparation information is already in the instance.)
action_this.revoke()
cleanup_prepare_files()
action_this = SchoolGradeInputAction(config_single)
action_this.run()
with db_context_manager as db_instance:
    result_single = list(db_instance.client_instance['demo']['school_grade_input'].find())
result_single_pd_correct_tz_3 = pd.DataFrame(get_correct_timezone(result_single))
result_single_pd_correct_tz_3

done clearing!
custom info from this action follows.




this is the DataSMART action for saving grade data for an exam.
Please modify school_grade_input_template.json to your need





done 1/1!
done!
remember to rm actions.demo.school_grade_input.prepare_result.p and actions.demo.school_grade_input.query_template.py if you want to start over for new action!


,_id,first_name,last_name,notes,score,subject,timestamp
0,57cf0f47aa9f2e116b0fd1f6,Emma,Apple,Animi facilis quaerat placeat necessitatibus i...,51,drawing,2006-07-21 20:53:19-04:00


## DST

notice the difference between UTC time, standard time (-05:00), and daylight saving time (-04:00). Internally, everything is UTC. We can try another one to see the difference between standard and DST.

In [14]:
action_this.revoke()
cleanup_prepare_files()
config_single = deepcopy(SchoolGradeInputAction().config)
config_single['batch_records'] = records[1:2]
action_this = SchoolGradeInputAction(config_single)
action_this.run()
with db_context_manager as db_instance:
    result_single = list(db_instance.client_instance['demo']['school_grade_input'].find())
result_single_pd_correct_tz_4 = pd.DataFrame(get_correct_timezone(result_single))
result_single_pd_correct_tz_4

done clearing!
custom info from this action follows.




this is the DataSMART action for saving grade data for an exam.
Please modify school_grade_input_template.json to your need





done 1/1!
done!
remember to rm actions.demo.school_grade_input.prepare_result.p and actions.demo.school_grade_input.query_template.py if you want to start over for new action!


,_id,first_name,last_name,score,subject,timestamp
0,57cf0f47aa9f2e116b0fd1fe,Liam,Banana,36,english,2006-02-02 10:48:06-05:00


## insert multiple records

multiple insertion is easy. simply make `batch_records` to have more than 1 record.

In [15]:
action_this.revoke()
cleanup_prepare_files()
config_multiple = deepcopy(SchoolGradeInputAction().config)
config_multiple['batch_records'] = records[:30]
action_this = SchoolGradeInputAction(config_multiple)
action_this.run()
with db_context_manager as db_instance:
    result_multiple = list(db_instance.client_instance['demo']['school_grade_input'].find())
result_multiple_pd = pd.DataFrame(get_correct_timezone(result_multiple))
result_multiple_pd

done clearing!
custom info from this action follows.




this is the DataSMART action for saving grade data for an exam.
Please modify school_grade_input_template.json to your need





done 1/30!
done 2/30!
done 3/30!
done 4/30!
done 5/30!
done 6/30!
done 7/30!
done 8/30!
done 9/30!
done 10/30!
done 11/30!
done 12/30!
done 13/30!
done 14/30!
done 15/30!
done 16/30!
done 17/30!
done 18/30!
done 19/30!
done 20/30!
done 21/30!
done 22/30!
done 23/30!
done 24/30!
done 25/30!
done 26/30!
done 27/30!
done 28/30!
done 29/30!
done 30/30!
done!
remember to rm actions.demo.school_grade_input.prepare_result.p and actions.demo.school_grade_input.query_template.py if you want to start over for new action!


,_id,first_name,last_name,notes,score,subject,timestamp
0,57cf0f47aa9f2e116b0fd206,Emma,Apple,Animi facilis quaerat placeat necessitatibus i...,51,drawing,2006-07-21 20:53:19-04:00
1,57cf0f47aa9f2e116b0fd207,Liam,Banana,NaN,36,english,2006-02-02 10:48:06-05:00
2,57cf0f47aa9f2e116b0fd208,Liam,Banana,Quae deserunt id deserunt consequatur sit.,12,music,2009-12-04 19:04:57-05:00
3,57cf0f47aa9f2e116b0fd209,Liam,Bush,NaN,45,math,2004-04-05 14:20:39-04:00
4,57cf0f47aa9f2e116b0fd20a,Olivia,Bush,Dignissimos ipsa consequuntur aut vitae iure o...,66,drawing,2009-01-22 12:03:51-05:00
5,57cf0f47aa9f2e116b0fd20b,Emma,Jobs,NaN,90,drawing,2006-01-07 20:42:40-05:00
6,57cf0f47aa9f2e116b0fd20c,Sophia,Apple,NaN,93,english,2007-05-21 13:45:55-04:00
7,57cf0f47aa9f2e116b0fd20d,Emily,Banana,NaN,18,english,2005-05-18 20:34:28-04:00
8,57cf0f47aa9f2e116b0fd20e,Liam,Bush,NaN,38,math,2010-05-23 16:18:45-04:00
9,57cf0f47aa9f2e116b0fd20f,Liam,Banana,NaN,56,music,2010-11-23 09:38:51-05:00


In [16]:
cleanup_prepare_files()